In [21]:
import numpy as np
import open3d as o3d
import struct
import matplotlib.pyplot as plt
from pandas import DataFrame
from pyntcloud import PyntCloud
import math
import random
#import Spectral as sp
from collections import defaultdict

In [22]:
def Bin_to_array(path):
    """_summary_

    Args:
        path (_type_): _description_

    Returns:
        _type_: _description_
    """
    points_row = []
    with open(path, 'rb') as f:
        content = f.read()
        iter = struct.iter_unpack('ffff', content)
        for idx, p in enumerate(iter):
            points_row.append([p[0], p[1], p[2]])
    return np.asarray(points_row, dtype=np.float32)

rootdir = '/home/daizj/Homeworks/cg/Proj/proj/data/training/velodyne/000011.bin'
points_row = Bin_to_array(rootdir)
points_pcd = o3d.geometry.PointCloud()
points_pcd.points = o3d.utility.Vector3dVector(points_row)
#pointcloud = points_row.reshape([3, -1])


In [23]:
def normal_filter(point_cloud):
    """_summary_

    Args:
        point_cloud (_type_): _description_
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud)
    pcd.estimate_normals(
        search_param=o3d.geometry.KDTreeSearchParamHybrid(
            radius=5.0, max_nn=9
        )
    )

    normals = np.asarray(pcd.normals)
    angular_distance_to_z = np.abs(normals[:, 2])
    idx_filtered = angular_distance_to_z > np.cos(np.pi/3)

    points_filtered= o3d.utility.Vector3dVector(point_cloud[idx_filtered])
    points_filtered=np.asarray(points_filtered)
    
    return points_filtered
    
points = normal_filter(points_row)
# points = points_row
print(points)
    

[[52.43999863  0.105       1.98199999]
 [51.90599823  0.26699999  1.96399999]
 [51.96500015  0.43099999  1.96599996]
 ...
 [ 3.68899989 -1.40100002 -1.72399998]
 [ 3.71300006 -1.398      -1.73399997]
 [ 3.70099998 -1.38       -1.72599995]]


In [24]:
def RANSAC_Seg(points, iters = 100, sigma = 0.2, outline_ratio = 0.5):
    """_summary_

    Args:
        points (_type_): _description_
        iters (int, optional): _description_. Defaults to 100.
        sigma (float, optional): _description_. Defaults to 0.2.
        outline_ratio (float, optional): _description_. Defaults to 0.5.

    Returns:
        _type_: _description_
    """
    segmented_idx,segmented_cloud = [],[]
    best_params = np.zeros((1,4)) #abcd

    preinlier = 0 
    probability = 0.99
    points_num = len(points)    
    
    for i in range(iters):
        ground_idx, ground_cloud = [], []
       
        sample_index = random.sample(range(points_num),3)   
        p1, p2, p3 = [points[i] for i in sample_index]
        
        norm = np.cross((p1-p2), (p1-p3))           
        a, b, c, d = norm[0], norm[1], norm[2], -np.dot(norm, p1)

        vec = (points - p1)    
        dist = abs(np.dot(vec, norm))/ np.linalg.norm(norm)   

        ground_idx = (dist <= sigma)
        inlier = np.sum(ground_idx == True)    
        if inlier > preinlier:                                         
            iters = math.log(1 - probability) / math.log(1 - pow(inlier / points_num, 3))  
            preinlier = inlier                                              
            best_params = [a,b,c,d]

        if inlier > points_num * (1-outline_ratio):
            break
        
    print("iters = %f" %iters)
    segmented_idx = np.logical_not(ground_idx)
    ground_cloud = points[ground_idx]
    # segmented_cloud = points[segmented_idx]
    return ground_cloud, best_params, ground_idx

ground_cloud, plane ,ground_idx = RANSAC_Seg(points, iters = 1000, sigma = 0.2, outline_ratio = 0.5)
print(plane)
ground = o3d.geometry.PointCloud()
ground.points = o3d.utility.Vector3dVector(ground_cloud)
ground.paint_uniform_color([0, 1.0, 0])
o3d.io.write_point_cloud("ground_objects.pcd",ground, write_ascii=True)
#print(segmented_idx)


iters = 12.430961
[2.2099806222754204, -2.3889889338667487, -113.90663314782957, -201.20380767094346]


True

In [25]:
def objCluster_dbscan(points, plane, ground):
        """_summary_

        Args:
            points (_type_): _description_
            plane (_type_): _description_

        Returns:
            _type_: _description_
        """
        plane, _ = ground.segment_plane(
            distance_threshold=0.05,
            ransac_n=3,
            num_iterations=1000)
        print(plane)
        distance_to_ground = np.abs(np.dot(points,np.asarray(plane[:3])) + plane[3])
        print(distance_to_ground)
        idx_cloud = distance_to_ground > 0.30
        
        segmented_objects = o3d.geometry.PointCloud()
        segmented_objects.points = o3d.utility.Vector3dVector(points[idx_cloud])

        labels = np.asarray(segmented_objects.cluster_dbscan(eps=0.60, min_points = 3))####
        k = labels.max()
        return labels, k, segmented_objects

obj_id, k, segmented_objects = objCluster_dbscan(points, plane, ground)
colors = plt.get_cmap("tab20")(obj_id / (k if k > 0 else 1))
colors[obj_id < 0] = 1
segmented_objects.colors = o3d.utility.Vector3dVector(colors[:, :3])

o3d.io.write_point_cloud("segmented_objects.pcd",segmented_objects, write_ascii=True)
print(len(segmented_objects.points))
print(k)
print(obj_id)


[-0.01660222  0.02516508  0.99954544  1.76514775]
[2.87826892 2.87321943 2.878366   ... 0.05457042 0.06488882 0.05624024]
17926
179
[  0   0   0 ... 179 179 179]


In [26]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.utils.data
from torch.autograd import Variable
from model import *

def predict_type(segmented_objects, obj_id, model_path):
    model = torch.load(model_path, map_location=torch.device('cuda'))
    classifier = PointNetCls(k = 4, num_points = 64, 
                             attention_type = 'Mn', feature_transform = 'store_true')
    
    classifier = classifier.eval()
    #classifier.load_state_dict(checkpoint['model_state_dict'])
    points = np.asarray(segmented_objects.points)
    points_with_id = np.hstack((points, obj_id.reshape(-1,1)))
    sorted_points_indices = np.argsort(points_with_id[:,3], axis=0).astype(np.int)
    points_with_id = points_with_id[sorted_points_indices]
    start_id = -1
    objects_indices = []
    for i in range(points_with_id.shape[0]):
        current_id = points_with_id[i,3]
        if current_id > start_id:
            start_id = current_id
            segmented_points = points[:i,:]
            print(segmented_points.shape)
            choice = np.random.choice(len(segmented_points), 64)
            segmented_points = segmented_points[choice]
            segmented_points = segmented_points.reshape(64,3,1)
            # print(segmented_points.shape)
            segmented_points = torch.from_numpy(segmented_points.astype(np.float32))
            # print(segmented_points.size())
            segmented_points = segmented_points.permute(2, 1, 0)
            segmented_points = segmented_points
            pred, _, _ = classifier(segmented_points)
            print(pred) 
            pred_choice = pred.data.max(1)[1]
            print(pred_choice, current_id-1)    

    return pred

model_path = "/home/daizj/Homeworks/cg/Proj/proj/model/KITTI_Mn_best_model.pth"
    
predict_type(segmented_objects, obj_id, model_path)

(182, 3)
tensor([[-1.2077, -1.6971, -1.2049, -1.5224]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) -1.0
(197, 3)
tensor([[-1.1697, -1.7991, -1.1359, -1.5947]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 0.0
(205, 3)
tensor([[-1.1684, -1.8133, -1.1314, -1.5925]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 1.0
(210, 3)
tensor([[-1.1687, -1.8147, -1.1281, -1.5960]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 2.0
(214, 3)
tensor([[-1.1685, -1.8150, -1.1282, -1.5959]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 3.0
(227, 3)
tensor([[-1.2116, -1.6871, -1.2157, -1.5109]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 4.0
(235, 3)
tensor([[-1.1712, -1.8136, -1.1224, -1.6023]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 5.0
(258, 3)
tensor([[-1.2156, -1.7067, -1.1717, -1.5503]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 6.0
(262, 3)
tensor([[-1.2056, -1.7315, -1.1691, -1.5473]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 7.0
(328, 3)
tensor([[-1.1928, -1.7899, -1.1217, -1.5900]], grad_fn=<LogSoft

/tmp/ipykernel_4944/2800286659.py:17: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sorted_points_indices = np.argsort(points_with_id[:,3], axis=0).astype(np.int)


tensor([[-1.0982, -1.8057, -1.1199, -1.7380]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 11.0
(4014, 3)
tensor([[-1.1592, -1.5925, -1.2061, -1.6957]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 12.0
(4021, 3)
tensor([[-0.8322, -3.2343, -0.8722, -2.2304]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 13.0
(4028, 3)
tensor([[-0.8443, -2.0985, -1.1728, -1.9804]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 14.0
(4037, 3)
tensor([[-0.9650, -1.9343, -1.1449, -1.8564]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 15.0
(4081, 3)
tensor([[-1.0783, -1.8680, -1.0895, -1.7777]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 16.0
(4086, 3)
tensor([[-1.2296, -1.5665, -1.2165, -1.5968]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 17.0
(4092, 3)
tensor([[-1.1135, -1.8480, -1.0961, -1.7156]], grad_fn=<LogSoftmaxBackward0>)
tensor([2]) 18.0
(4096, 3)
tensor([[-0.6278, -3.5181, -1.0104, -2.6239]], grad_fn=<LogSoftmaxBackward0>)
tensor([0]) 19.0
(4120, 3)
tensor([[-0.9028, -1.9233, -1.2077, -1.9001]], grad_fn=

tensor([[-1.1741, -1.7767, -1.0966, -1.6728]], grad_fn=<LogSoftmaxBackward0>)